In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import tensorflow as tf
#import random
import numpy as np
#random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)


In [2]:
#upload csv file
from google.colab import files

uploaded = files.upload()


Saving HR_Analytics_clean.csv to HR_Analytics_clean.csv


In [3]:

#  Import and read the csv.

hr_dummies_df= pd.read_csv("HR_Analytics_clean.csv")
hr_dummies_df

,AgeGroup,Attrition,BusinessTravel,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobSatisfaction,...,YearsWithCurrManager,Sales,Human_Resources,Research_&_Development,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree
0,1,1,1,3,3,3,1,3,1,3,...,0.0,0,0,2,False,True,False,False,False,False
1,1,0,1,10,3,4,0,2,1,3,...,0.0,1,0,0,False,False,False,True,False,False
2,1,1,2,5,3,2,1,3,1,2,...,0.0,1,0,0,False,False,True,False,False,False
3,1,0,3,5,2,2,1,3,1,4,...,0.0,0,0,3,False,True,False,False,False,False
4,1,1,3,8,1,3,1,3,1,3,...,0.0,0,0,2,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,5,0,1,7,3,1,0,3,5,1,...,10.0,0,0,4,False,True,False,False,False,False
1412,5,0,2,28,3,3,0,2,3,1,...,11.0,3,0,0,False,False,True,False,False,False
1413,5,0,1,16,4,1,1,3,2,1,...,2.0,3,0,0,False,False,True,False,False,False
1414,5,0,1,7,4,2,1,4,2,4,...,9.0,3,0,0,False,False,True,False,False,False


In [4]:
# Split our preprocessed data into our features and target arrays
X = hr_dummies_df.drop(["Attrition"], axis='columns')
y = hr_dummies_df["Attrition"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,random_state=78)

In [5]:
#normalize the data
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [7]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2800      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5261 (20.55 KB)
Trainable params: 5261 (20.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
36/36 [==============================] - 1s 2ms/step - loss: 0.4711 - accuracy: 0.8277
Epoch 2/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3839 - accuracy: 0.8489
Epoch 3/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3417 - accuracy: 0.8631
Epoch 4/100
36/36 [==============================] - 0s 2ms/step - loss: 0.3111 - accuracy: 0.8887
Epoch 5/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2877 - accuracy: 0.8922
Epoch 6/100
36/36 [==============================] - 0s 3ms/step - loss: 0.2712 - accuracy: 0.8913
Epoch 7/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2572 - accuracy: 0.9019
Epoch 8/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2449 - accuracy: 0.9055
Epoch 9/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2322 - accuracy: 0.9125
Epoch 10/100
36/36 [==============================] - 0s 2ms/step - loss: 0.2219 - accuracy: 0.9108
Epoch 11/

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 1.2351 - accuracy: 0.8415 - 259ms/epoch - 29ms/step
Loss: 1.2351055145263672, Accuracy: 0.841549277305603


Optimization #1
- Reduce Epochs to 39


In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2800      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5261 (20.55 KB)
Trainable params: 5261 (20.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
# Train the model
fit_model = nn1.fit(X_train_scaled, y_train, epochs=39)

Epoch 1/39
36/36 [==============================] - 2s 2ms/step - loss: 0.6028 - accuracy: 0.6776
Epoch 2/39
36/36 [==============================] - 0s 2ms/step - loss: 0.3997 - accuracy: 0.8392
Epoch 3/39
36/36 [==============================] - 0s 2ms/step - loss: 0.3551 - accuracy: 0.8534
Epoch 4/39
36/36 [==============================] - 0s 2ms/step - loss: 0.3204 - accuracy: 0.8675
Epoch 5/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2936 - accuracy: 0.8878
Epoch 6/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2781 - accuracy: 0.8869
Epoch 7/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2636 - accuracy: 0.9002
Epoch 8/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2517 - accuracy: 0.9028
Epoch 9/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2413 - accuracy: 0.9090
Epoch 10/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2292 - accuracy: 0.9117
Epoch 11/39
36/36 [

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.7100 - accuracy: 0.8345 - 181ms/epoch - 20ms/step
Loss: 0.7099973559379578, Accuracy: 0.8345070481300354


Optimization #2
- Add a 3rd hidden layer

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
hidden_layer_3 = 10

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_layer_3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                2800      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5551 (21.68 KB)
Trainable params: 5551 (21.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=39)

Epoch 1/39
36/36 [==============================] - 2s 4ms/step - loss: 0.4749 - accuracy: 0.8383
Epoch 2/39
36/36 [==============================] - 0s 3ms/step - loss: 0.3935 - accuracy: 0.8383
Epoch 3/39
36/36 [==============================] - 0s 4ms/step - loss: 0.3542 - accuracy: 0.8383
Epoch 4/39
36/36 [==============================] - 0s 4ms/step - loss: 0.3252 - accuracy: 0.8383
Epoch 5/39
36/36 [==============================] - 0s 4ms/step - loss: 0.3026 - accuracy: 0.8454
Epoch 6/39
36/36 [==============================] - 0s 4ms/step - loss: 0.2864 - accuracy: 0.8595
Epoch 7/39
36/36 [==============================] - 0s 4ms/step - loss: 0.2703 - accuracy: 0.8746
Epoch 8/39
36/36 [==============================] - 0s 4ms/step - loss: 0.2539 - accuracy: 0.8931
Epoch 9/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2383 - accuracy: 0.9055
Epoch 10/39
36/36 [==============================] - 0s 2ms/step - loss: 0.2200 - accuracy: 0.9125
Epoch 11/39
36/36 [

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 1.1724 - accuracy: 0.8345 - 148ms/epoch - 16ms/step
Loss: 1.172446370124817, Accuracy: 0.8345070481300354


Optimization #3
- Reduce epochs to 25- just before accuracy was 1 & the machine had perfectly learned the model

In [18]:
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
hidden_layer_3 = 10

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_layer_3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                2800      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 10)                310       
                                                                 
 dense_13 (Dense)            (None, 1)                 11        
                                                                 
Total params: 5551 (21.68 KB)
Trainable params: 5551 (21.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
36/36 [==============================] - 1s 2ms/step - loss: 0.4776 - accuracy: 0.8366
Epoch 2/25
36/36 [==============================] - 0s 2ms/step - loss: 0.3897 - accuracy: 0.8383
Epoch 3/25
36/36 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.8401
Epoch 4/25
36/36 [==============================] - 0s 2ms/step - loss: 0.3193 - accuracy: 0.8578
Epoch 5/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2876 - accuracy: 0.8860
Epoch 6/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2645 - accuracy: 0.8949
Epoch 7/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2475 - accuracy: 0.9046
Epoch 8/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2332 - accuracy: 0.9090
Epoch 9/25
36/36 [==============================] - 0s 3ms/step - loss: 0.2165 - accuracy: 0.9178
Epoch 10/25
36/36 [==============================] - 0s 3ms/step - loss: 0.1998 - accuracy: 0.9205
Epoch 11/25
36/36 [

In [21]:
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.6883 - accuracy: 0.8345 - 143ms/epoch - 16ms/step
Loss: 0.6882864832878113, Accuracy: 0.8345070481300354


Optimization #4
- Change 2 hidden layer to 'tanh'

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
hidden_layer_3 = 10

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="tanh"))

# third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 80)                2800      
                                                                 
 dense_15 (Dense)            (None, 30)                2430      
                                                                 
 dense_16 (Dense)            (None, 80)                2480      
                                                                 
 dense_17 (Dense)            (None, 1)                 81        
                                                                 
Total params: 7791 (30.43 KB)
Trainable params: 7791 (30.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn4.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
36/36 [==============================] - 1s 2ms/step - loss: 0.3728 - accuracy: 0.8463
Epoch 2/25
36/36 [==============================] - 0s 2ms/step - loss: 0.3201 - accuracy: 0.8728
Epoch 3/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2926 - accuracy: 0.8869
Epoch 4/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2753 - accuracy: 0.8887
Epoch 5/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2526 - accuracy: 0.9037
Epoch 6/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2383 - accuracy: 0.9072
Epoch 7/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2287 - accuracy: 0.9099
Epoch 8/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2107 - accuracy: 0.9214
Epoch 9/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1942 - accuracy: 0.9267
Epoch 10/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1791 - accuracy: 0.9302
Epoch 11/25
36/36 [

In [27]:
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.9993 - accuracy: 0.8169 - 154ms/epoch - 17ms/step
Loss: 0.9992536306381226, Accuracy: 0.8169013857841492


Optimization #5
- change optimizer to 'RMSprop'

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
hidden_layer_3 = 10

nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="tanh"))

# third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu"))

# Output layer
nn5.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn5.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 80)                2800      
                                                                 
 dense_19 (Dense)            (None, 30)                2430      
                                                                 
 dense_20 (Dense)            (None, 80)                2480      
                                                                 
 dense_21 (Dense)            (None, 1)                 81        
                                                                 
Total params: 7791 (30.43 KB)
Trainable params: 7791 (30.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# Compile the model
nn5.compile(loss="binary_crossentropy", optimizer="RMSprop", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn5.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
36/36 [==============================] - 0s 4ms/step - loss: 0.2293 - accuracy: 0.9161
Epoch 2/25
36/36 [==============================] - 0s 4ms/step - loss: 0.2167 - accuracy: 0.9205
Epoch 3/25
36/36 [==============================] - 0s 2ms/step - loss: 0.2015 - accuracy: 0.9258
Epoch 4/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1903 - accuracy: 0.9249
Epoch 5/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1779 - accuracy: 0.9320
Epoch 6/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1683 - accuracy: 0.9390
Epoch 7/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1530 - accuracy: 0.9514
Epoch 8/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1427 - accuracy: 0.9541
Epoch 9/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1283 - accuracy: 0.9576
Epoch 10/25
36/36 [==============================] - 0s 2ms/step - loss: 0.1177 - accuracy: 0.9602
Epoch 11/25
36/36 [

In [32]:
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 1.1272 - accuracy: 0.8063 - 167ms/epoch - 19ms/step
Loss: 1.127150058746338, Accuracy: 0.8063380122184753


Optimization #6
-Change optimizer to 'adagrad' because it's effective for training models with sparse data or high-dimensional feature spaces

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 80
hidden_layer_2 = 30
hidden_layer_3 = 10

nn6 = tf.keras.models.Sequential()

# First hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="tanh"))

# third hidden layer
nn6.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu"))


# Output layer
nn6.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn6.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 80)                2800      
                                                                 
 dense_27 (Dense)            (None, 30)                2430      
                                                                 
 dense_28 (Dense)            (None, 80)                2480      
                                                                 
 dense_29 (Dense)            (None, 1)                 81        
                                                                 
Total params: 7791 (30.43 KB)
Trainable params: 7791 (30.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn6.compile(loss="binary_crossentropy", optimizer="adagrad", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn6.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
36/36 [==============================] - 1s 2ms/step - loss: 0.5633 - accuracy: 0.7827
Epoch 2/25
36/36 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 0.8233
Epoch 3/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4989 - accuracy: 0.8348
Epoch 4/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4812 - accuracy: 0.8392
Epoch 5/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4680 - accuracy: 0.8383
Epoch 6/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4580 - accuracy: 0.8383
Epoch 7/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4502 - accuracy: 0.8383
Epoch 8/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4439 - accuracy: 0.8383
Epoch 9/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4390 - accuracy: 0.8383
Epoch 10/25
36/36 [==============================] - 0s 2ms/step - loss: 0.4347 - accuracy: 0.8383
Epoch 11/25
36/36 [

In [38]:
model_loss, model_accuracy = nn6.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

9/9 - 0s - loss: 0.4247 - accuracy: 0.8380 - 175ms/epoch - 19ms/step
Loss: 0.4247158467769623, Accuracy: 0.8380281925201416


Optimization Attempts Table

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 100
hidden_layer_2 = 60
hidden_layer_3 = 30

nn7 = tf.keras.models.Sequential()

# First hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="tanh"))

# third hidden layer
nn7.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu"))


# Output layer
nn7.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn7.summary()

# compile the model
nn7.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = nn7.fit(X_train_scaled, y_train, epochs=100, verbose=2)

# evaluate the model
loss, accuracy, f1_score, precision, recall = nn7.evaluate(X_test_scaled, y_test, verbose=0)

print(f"Loss: {loss}, Accuracy: {accuracy}, f1_score: {f1_score}, Precision: {precision}, Recall: {recall}")
#f1 tells us the employees who would leave

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 100)               3500      
                                                                 
 dense_29 (Dense)            (None, 60)                6060      
                                                                 
 dense_30 (Dense)            (None, 100)               6100      
                                                                 
 dense_31 (Dense)            (None, 1)                 101       
                                                                 
Total params: 15761 (61.57 KB)
Trainable params: 15761 (61.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
36/36 - 1s - loss: 0.4495 - acc: 0.8207 - f1_m: 0.0415 - precision_m: 0.0484 - recall_m: 0.0516 - 1s/epoch - 34ms/step
Epoch 2/100
36/36 -